In [1]:
# script to handle more complicated cases of rotors not completing or the peaks/valleys issues
import glob
import sys
import os
import numpy as np
import shutil

import ase.io.gaussian

import logging
import arkane.main
import autotst.species

sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/dft/')
sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/databse/')
import autotst_wrapper
import database_fun

/home/harris.se/rmg/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: [Errno 2] No such file or directory: 'julia': 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


In [2]:
# autotst_wrapper.run_rotor_offset(75, 4)
from arkane.ess import _registered_ess_adapters
from arkane.ess import ess_factory
from arkane.modelchem import standardize_name
from arkane.statmech import ScanLog as ScanLog
from arkane.modelchem import LevelOfTheory
from arkane.modelchem import CompositeLevelOfTheory

In [3]:
# species_index = 369
# rotor_index = 0
# autotst_wrapper.run_rotor_offset(species_index, rotor_index)

In [4]:
def hinderedRotor(scanLog, pivots, top, symmetry=None, fit='best'):
    """Read a hindered rotor directive, and return the attributes in a list"""
    return [scanLog, pivots, top, symmetry, fit]

def hinderedRotor1DArray(angles, energies, pivots, top, symmetry=None, fit='best'):
    """Read a hindered rotor PES profile, and return the attributes in a list"""
    return [angles, energies, pivots, top, symmetry, fit]

def freeRotor(pivots, top, symmetry):
    """Read a free rotor directive, and return the attributes in a list"""
    return [pivots, top, symmetry]


def hinderedRotor2D(scandir, pivots1, top1, symmetry1, pivots2, top2, symmetry2, symmetry='none'):
    """Read a two dimensional hindered rotor directive, and return the attributes in a list"""
    return [scandir, pivots1, top1, symmetry1, pivots2, top2, symmetry2, symmetry]


def hinderedRotorClassicalND(calc_path, pivots, tops, sigmas, semiclassical):
    """Read an N dimensional hindered rotor directive, and return the attributes in a list"""
    return [calc_path, pivots, tops, sigmas, semiclassical]


def create_log(log_path, check_for_errors=True):
    if not os.path.isfile(log_path):
        modified_log_path = os.path.join(directory, log_path)
        if not os.path.isfile(modified_log_path):
            raise InputError('Could not find log file for species {0} '
                             'in the specified path {1}'.format(self.species.label, log_path))
        else:
            log_path = modified_log_path

    return ess_factory(log_path, check_for_errors=check_for_errors)


In [5]:
def delete_rotor(conformer_py_file, rotor_index):
    with open(conformer_py_file, 'r') as f:
        file_lines = f.readlines()

    # actually comments out the line with that rotor index
    for i, line in enumerate(file_lines):
        if f'rotor_{rotor_index:04}.log' in line:
            print(f'Found line with rotor {rotor_index}')
            if line.startswith('#'):
                print('Rotor already commented out')
            else:
                file_lines[i] = '# ' + line
                with open(conformer_py_file, 'w') as f:
                    f.writelines(file_lines)
            return

    print('Rotor index not found')


In [36]:
species_index = 114
sp_done = autotst_wrapper.arkane_species_complete(species_index)
print(sp_done)
assert not sp_done

False


In [37]:
N_rotors = -1
rotor_scan_completed = []
rotor_scan_incomplete = []
rotor_scan_works = []
rotors_to_delete = []

rotors_turned_off = []  # will later comment out the lines here to make the species run


# Copy files
# check if the arkane folder has been set up???
species_dir = os.path.join(autotst_wrapper.DFT_DIR, 'thermo', f'species_{species_index:04}')
conformer_dir = os.path.join(species_dir, 'conformers')
rotor_dir = os.path.join(species_dir, 'rotors')
arkane_dir = os.path.join(species_dir, 'arkane')
splice_dir = os.path.join(rotor_dir, f'spliced_rotors')
os.makedirs(arkane_dir, exist_ok=True)
os.chdir(arkane_dir)


if autotst_wrapper.arkane_species_complete(species_index):
    species_log(species_index, f'Arkane species already complete')

else:
    rmg_species = database_fun.index2species(species_index)
    species_smiles = rmg_species.smiles

    new_cf = autotst.species.Conformer(smiles=species_smiles)


    # copy the conformer file in the rotors dir
    conformer_files = glob.glob(os.path.join(rotor_dir, 'conformer_*.log'))
    assert conformer_files, 'No conformer files in rotor dir'
    conformer_file = conformer_files[0]
    shutil.copy(conformer_file, arkane_dir)

    # copy the rotor files
    rotor_files = glob.glob(os.path.join(rotor_dir, 'rotor_*.log'))
    N_rotors = len(rotor_files)
    for rotor_index, rotor_file in enumerate(rotor_files):
        # check if the rotor file completed
        if autotst_wrapper.get_termination_status(rotor_file) == 0:
            shutil.copy(rotor_file, arkane_dir)
            rotor_scan_completed.append(rotor_index)
        else:
            # check if we've alread run the offset rotors
            rotor_scan_incomplete.append(rotor_index)
            if not os.path.exists(os.path.join(splice_dir, f'rotor_{rotor_index:04}_fwd_0000.log')):
                # don't do this actually
                print('not going to run rotors at offsets...')
                pass
#                 print('Running rotors at offsets')
#                 autotst_wrapper.run_rotor_offset(species_index, rotor_index)
            else:
                print(f'Rotor {rotor_index} did not complete')
                
# if some rotors didn't complete... first try replacing them with any offsets
rotors_to_delete = []
for rotor_index in rotor_scan_incomplete:
    print(f'Rotor {rotor_index}')
    complete_scans = []
    rotor_offset_files = glob.glob(os.path.join(splice_dir, f'rotor_{rotor_index:04}_fwd_*.log'))
#     print(rotor_offset_files)
    for i, r_file in enumerate(rotor_offset_files):
        if autotst_wrapper.get_termination_status(r_file) == 0:
            complete_scans.append(i)
    if not complete_scans:
        print('No scans completed, rotor must be deleted')
        rotors_to_delete.append(rotor_index)
    else:
        print('Some offset scans completed, will be copied or deleted later')
    print()

In [38]:
# get the main conformer
with open(conformer_file, 'r') as f:
    atoms = ase.io.gaussian.read_gaussian_out(f)

new_cf._ase_molecule = atoms
new_cf.update_coords_from(mol_type="ase")

torsions = new_cf.get_torsions()
assert len(torsions) == len(rotor_files)

# write the Arkane conformer file
autotst_wrapper.write_arkane_conformer_file(new_cf, conformer_file, arkane_dir, include_rotors=True)

conformer_py_file = os.path.basename(conformer_file[:-4]) + '.py'

# write the Arkane input file
input_file = os.path.join(arkane_dir, 'input.py')
formula = new_cf.rmg_molecule.get_formula()
lines = [
    '#!/usr/bin/env python\n\n',
    f'modelChemistry = "M06-2X/cc-pVTZ"\n',
    'useHinderedRotors = True\n',
    'useBondCorrections = False\n\n',

    'frequencyScaleFactor = 0.982\n',

    f"species('{formula}', '{conformer_py_file}', structure=SMILES('{new_cf.rmg_molecule.smiles}'))\n\n",

    f"thermo('{formula}', 'NASA')\n",
]
with open(input_file, 'w') as f:
    f.writelines(lines)


In [39]:
# delete bad rotors
rotors_to_delete.append(0)
for r in rotors_to_delete:
    delete_rotor(conformer_py_file, r)
        

Rotor index not found


In [40]:
# run once to populate the arkane_species object
arkane_species = arkane.main.Arkane(input_file=input_file, output_directory=arkane_dir, verbose=logging.DEBUG)
arkane_species.plot = True
arkane_species.execute()  # TODO don't forget to run the arkane job[1]

Arkane execution initiated at Tue Apr  9 11:49:05 2024

################################################################
#                                                              #
# Automated Reaction Kinetics and Network Exploration (Arkane) #
#                                                              #
#   Version: 3.2.0                                             #
#   Authors: RMG Developers (rmg_dev@mit.edu)                  #
#   P.I.s:   William H. Green (whgreen@mit.edu)                #
#            Richard H. West (r.west@neu.edu)                  #
#   Website: http://reactionmechanismgenerator.github.io/      #
#                                                              #
################################################################

The current git HEAD for RMG-Py is:
	51d7f0c661a7ae4519c7033d38d0be97e75900d5
	Wed Dec 13 14:48:41 2023 -0500

The current git HEAD for RMG-database is:
	cfb4910cfc64274981216acbfb7756aba6be0112
	Tue Dec 12 11:55:13 2023 -0500



*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


LogError: There was an error (Internal coordinate error) with Gaussian output file conformer_0002.log due to line:
 Error termination via Lnk1e in /shared/centos7/gaussian/g16/l103.exe at Tue Jan  9 00:25:47 2024.


In [41]:
# try running in a loop
offset_attempt = np.array(np.zeros(N_rotors), int)

for overall_loop_index in range(0, 10):
    print(f' --------------------- Loop {overall_loop_index} ----------------')
    error_text = ''
    try:
        arkane_species.job_list[0].execute()
        print('Arkane worked! Exiting')
        break
    except Exception as myerror:
        error_text = str(myerror)
    
    global_context = {
        '__builtins__': None,
    }
    local_context = {
        '__builtins__': None,
        'True': True,
        'False': False,
        'HinderedRotor': hinderedRotor,
        'HinderedRotor1DArray': hinderedRotor1DArray,
        'FreeRotor': freeRotor,
        'HinderedRotor2D': hinderedRotor2D,
        'HinderedRotorClassicalND': hinderedRotorClassicalND,
        'ScanLog': ScanLog,
        'Log': create_log,  # The Log class no longer exists, so route the path to ess_factory instead
        'LevelOfTheory': LevelOfTheory,
        'CompositeLevelOfTheory': CompositeLevelOfTheory,
    }

    local_context.update({ess_adapter_name: create_log for ess_adapter_name in _registered_ess_adapters.keys()})

    # ----------------- Identify the problem rotor and delete or copy it ------------------------
    with open(arkane_species.job_list[0].path, 'r') as f:
        try:
            exec(f.read(), global_context, local_context)
        except:
            logging.error('The species file {0} was invalid:'.format(arkane_species.job_list[0].path))
            raise
            
    for i, rotor in enumerate(local_context['rotors']):
        rotor_index = int(rotor[0].path[-8:-4])
        print(rotor_index)
        if str(rotor[1]) in error_text:
            print(f'rotor {rotor_index} failed')
            print(rotor[0], rotor[1], rotor[2])

            # see if there are any valid replacements in the spliced folder:
            complete_scans = []
            rotor_offset_files = glob.glob(os.path.join(splice_dir, f'rotor_{rotor_index:04}_fwd_*.log'))
            for j, r_file in enumerate(rotor_offset_files):
                if autotst_wrapper.get_termination_status(r_file) == 0:
                    complete_scans.append(j)
            if not complete_scans:
                print(f'No scans completed, rotor {rotor_index} must be deleted')
                rotors_to_delete.append(rotor_index)
                delete_rotor(conformer_py_file, rotor_index)
            else:
                # find the best rotor to copy over
                my_offset_files = glob.glob(os.path.join(splice_dir, f'rotor_{rotor_index:04}_fwd_*.log'))
                possible_files = []
                possible_e0s = []
                for k, fname in enumerate(my_offset_files):
                    if autotst_wrapper.get_termination_status(fname) != 0:
                        continue
                    print(fname)
                    my_log2 = arkane.ess.gaussian.GaussianLog(fname)
                    e, a = my_log2.load_scan_energies()
                    if not np.isclose(e[0], e[-1]):
                        continue

                    possible_files.append(fname)
                    possible_e0s.append(e[0])

                # rank the files by energy, picking the lowest energy, with the reasoning that a scan
                # which starts at a low energy is likely to start out in a stable configuration, so
                # if it makes it all the way around back to a stable energy, then it's more trustworthy
                indices = np.arange(0, len(possible_files))
                order = [x for _, x in sorted(zip(possible_e0s, indices))]
                
                if offset_attempt[rotor_index] == len(possible_files):
                    print(f'No more offset attempts to try, rotor {rotor_index} must be deleted')
                    delete_rotor(conformer_py_file, rotor_index)
                else:
                    print(f'This is attempt {offset_attempt[rotor_index]} on rotor {rotor_index}')
                    print(f'Trying rotor offset scan {order[offset_attempt[rotor_index]]} completed. Copying...')
                    print(possible_files[order[offset_attempt[rotor_index]]])
                    shutil.copy(possible_files[order[offset_attempt[rotor_index]]], os.path.join(arkane_dir, f'rotor_{rotor_index:04}.log'))
                    offset_attempt[rotor_index] += 1                                      
                                                                  
            break

 --------------------- Loop 0 ----------------
Loading statistical mechanics parameters for C3H3O...
Error: The species file /work/westgroup/harris.se/autoscience/reaction_calculator/dft/thermo/species_0114/arkane/conformer_0002.py was invalid:


LogError: There was an error (Internal coordinate error) with Gaussian output file conformer_0002.log due to line:
 Error termination via Lnk1e in /shared/centos7/gaussian/g16/l103.exe at Tue Jan  9 00:25:47 2024.


In [35]:
arkane_species.execute()  # TODO don't forget to run the arkane job[1]

Arkane execution initiated at Tue Apr  9 11:47:48 2024

################################################################
#                                                              #
# Automated Reaction Kinetics and Network Exploration (Arkane) #
#                                                              #
#   Version: 3.2.0                                             #
#   Authors: RMG Developers (rmg_dev@mit.edu)                  #
#   P.I.s:   William H. Green (whgreen@mit.edu)                #
#            Richard H. West (r.west@neu.edu)                  #
#   Website: http://reactionmechanismgenerator.github.io/      #
#                                                              #
################################################################

The current git HEAD for RMG-Py is:
	51d7f0c661a7ae4519c7033d38d0be97e75900d5
	Wed Dec 13 14:48:41 2023 -0500

The current git HEAD for RMG-database is:
	cfb4910cfc64274981216acbfb7756aba6be0112
	Tue Dec 12 11:55:13 2023 -0500



*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


Dumping species C2H5O data as species/C2H5O.yml
Assigning font /b'F1' = '/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'


*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)


Embedding font /work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
Writing TrueType font.

Arkane execution terminated at Tue Apr  9 11:47:49 2024
